In [35]:
import pdb
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

import numpy as np

import io
import os
import json

# https://github.com/bolaurent/zuora_restful_python
from zuora_restful_python.zuora import Zuora

ZUORA_CONFIG_SANDBOX = json.load(open(os.path.expanduser('~') + '/.zuora-sandbox-config.json'))


zuora_sandbox = Zuora(ZUORA_CONFIG_SANDBOX['user'],ZUORA_CONFIG_SANDBOX['password'],endpoint='sandbox')


FIELDS = [
    'RatePlanCharge.DTCV'
    ,'RecognizedRevenueAccountingCode.Name'
    ,'Subscription.Name'
    ,'Subscription.SubscriptionStartDate'
]

# given a date like '2017-01-02', return a period name like '2017-01-01'
def convert_date_to_periodname(d):
    return '{}-{}'.format(d[0:4],d[5:7])

# query rateplancharges and return a pandas dataframe, indexed on subscription name
def read_rateplancharges(zuora):
    q = """
        SELECT {} FROM RatePlanCharge 
        WHERE Subscription.SubscriptionStartDate >= '2017-04-01'
        AND Subscription.SubscriptionStartDate < '2017-07-01'
        """.format(','.join(FIELDS))
    csvData = zuora.query_export(q)
    df = pd.read_csv(io.StringIO(csvData), index_col=['Subscription.Name'])
    
    return df

# query rateplancharges from zuora
rateplancharges = read_rateplancharges(zuora_sandbox)


# create a column that shows the fiscal period, based on the subscription start date
rateplancharges['Month'] = rateplancharges['Subscription.SubscriptionStartDate'].apply(convert_date_to_periodname)

# pivot the table to show total revenue by account and period
rateplancharges = pd.pivot_table(rateplancharges, index=['RecognizedRevenueAccountingCode.Name'], \
                                 columns=['Month'], aggfunc=np.sum, values=['RatePlanCharge.DTCV'])

# Obfuscate the data for publication
rateplancharges = rateplancharges.applymap(lambda x: np.random.rand() * 100000000)


# print it in a nice table
rateplancharges




RatePlanCharge.DTCV                \
Month                                            2017-04       2017-05   
RecognizedRevenueAccountingCode.Name                                     
Incidental Revenue                         14,964,891.49 73,997,740.37   
Licensing Revenue                          54,753,750.22 48,986,764.30   
Marketplace Revenue                        24,389,988.39  1,126,351.16   
Operations Revenue                         64,868,916.15 41,952,120.78   
Professional Services Revenue              56,580,822.22 24,017,393.28   
Support Revenue                            82,891,468.69 44,856,189.27   

                                                    
Month                                      2017-06  
RecognizedRevenueAccountingCode.Name                
Incidental Revenue                   12,936,659.63  
Licensing Revenue                    42,942,669.71  
Marketplace Revenue                  76,601,782.67  
Operations Revenue                   37,441,958.23  
Professional Services Revenue        31,667,894.23  
Support Revenue                       6,112,021.05